In [1]:
import requests
import json
import pandas as pd
import numpy as np

In [2]:
## function for getting game ids for a season 

def get_schedule(season):
    schedule_list = []
    url = "https://live.nhl.com/GameData/SeasonSchedule-" + str(season) + ".json"
    schedule = requests.get(url)
    for i in range(len(schedule.json())):
        gameid = schedule.json()[i]['id']
        datetime = schedule.json()[i]['est']
        away = schedule.json()[i]['a']
        home = schedule.json()[i]['h']
        NHL_schedule = pd.DataFrame([[gameid,datetime,away,home]],
                              columns = ['game_id', 'date_time', 'away', 'home'])
        schedule_list.append(NHL_schedule)
        
    schedule_df = pd.concat(schedule_list)
    return schedule_df


In [31]:
## function for getting all events in a game

def get_data(gameid):
    game_events = []
    num = 0
    url = "https://api-web.nhle.com/v1/gamecenter/" + str(gameid) + "/play-by-play"
    response = requests.get(url)
    gametype = response.json()['gameType']
    venue = response.json()['venue']['default']
    starttimeutc = response.json()['startTimeUTC']
    awayid = response.json()['awayTeam']['id']
    awayname = response.json()['awayTeam']['name']['default']
    awayabr = response.json()['awayTeam']['abbrev']
    awayscore = response.json()['awayTeam']['score']
    awaysog = response.json()['awayTeam']['sog']
    homeid = response.json()['homeTeam']['id']
    homename = response.json()['homeTeam']['name']['default']
    homeabr = response.json()['homeTeam']['abbrev']
    homescore = response.json()['homeTeam']['score']
    homesog = response.json()['homeTeam']['sog']
    ot_reg = response.json()['gameOutcome']['lastPeriodType']
    for i in response.json()['plays']:
        num = num + 1
        eventid = i['eventId']
        period = i['periodDescriptor']['number']
        period_type = i['periodDescriptor']['periodType']
        try:
            sit_code = i['situationCode']
        except KeyError as error:
            pass
        timeinperiod = i['timeInPeriod']
        timeremaining = i['timeRemaining']
        hometeamdefside = i['homeTeamDefendingSide']
        typecode = i['typeCode']
        typekey = i['typeDescKey']
        if i['typeCode'] in (502, 503, 504, 505, 506, 507, 508, 525, 537):
            eventownerteamid = i['details']['eventOwnerTeamId']
            xcoord = i['details']['xCoord']
            ycoord = i['details']['yCoord']
            zone = i['details']['zoneCode']
            descKey = 'NA'
            homeplayer = 'NA'
            awayplayer = 'NA'
            event_row = pd.DataFrame([[gameid, gametype, venue, starttimeutc, awayid, awayname, awayabr, awayscore,
                                      awaysog, homeid, homename, homeabr, homescore, homesog, ot_reg,
                                      num, eventid, period, period_type, sit_code, timeinperiod, timeremaining,
                                       hometeamdefside, typecode, typekey, eventownerteamid,
                                      xcoord, ycoord, zone, descKey, homeplayer, awayplayer]],
                              columns = ['game_id','game_type', 'venue', 'start_time_utc', 'away_team_id','away_team_name',
                                        'away_team_abbrev','away_score','away_sog', 'home_team_id', 'home_team_name',
                                        'home_team_abbrev', 'home_score', 'home_sog','ot_reg', 'event_number',
                                        'event_id','period_number', 'period_type', 'situation_code', 'time_in_period',
                                        'time_remaining_in_period', 'home_team_defending_side',
                                        'event_type_code','event_type_key', 'event_owner_team_id',
                                        'x_coordinate', 'y_coordinate', 'zone', 'description_key',
                                        'home_player', 'away_player'])
            game_events.append(event_row)
        
        elif i['typeCode'] == 509:
            eventownerteamid = i['details']['eventOwnerTeamId']
            xcoord = i['details']['xCoord']
            ycoord = i['details']['yCoord']
            zone = i['details']['zoneCode']
            descKey = i['details']['descKey']
            try:
                if awayid == eventownerteamid:
                        awayplayer = i['details']['committedByPlayerId']
                        homeplayer = i['details']['drawnByPlayerId']
                elif homeid == eventownerteamid:
                    awayplayer = i['details']['drawnByPlayerId']
                    homeplayer = i['details']['committedByPlayerId']
            except KeyError as error:
                awayplayer = 'NA'
                homeplayer = 'NA'
            event_row = pd.DataFrame([[gameid, gametype, venue, starttimeutc, awayid, awayname, awayabr, awayscore,
                                      awaysog, homeid, homename, homeabr, homescore, homesog, ot_reg,
                                      num, eventid, period, period_type, sit_code, timeinperiod, timeremaining,
                                       hometeamdefside, typecode, typekey, eventownerteamid,
                                      xcoord, ycoord, zone, descKey, homeplayer, awayplayer]],
                              columns = ['game_id','game_type', 'venue', 'start_time_utc', 'away_team_id','away_team_name',
                                        'away_team_abbrev','away_score','away_sog', 'home_team_id', 'home_team_name',
                                        'home_team_abbrev', 'home_score', 'home_sog','ot_reg', 'event_number',
                                        'event_id','period_number', 'period_type', 'situation_code', 'time_in_period',
                                        'time_remaining_in_period', 'home_team_defending_side',
                                        'event_type_code','event_type_key', 'event_owner_team_id',
                                        'x_coordinate', 'y_coordinate', 'zone', 'description_key',
                                        'home_player', 'away_player'])
            game_events.append(event_row)
            
        elif i['typeCode'] in (516,520,521,523,524):
            eventownerteamid = 'NA'
            xcoord = 'NA'
            ycoord = 'NA'
            zone = 'NA'
            descKey = 'NA'
            homeplayer = 'NA'
            awayplayer = 'NA'
            event_row = pd.DataFrame([[gameid, gametype, venue, starttimeutc, awayid, awayname, awayabr, awayscore,
                                      awaysog, homeid, homename, homeabr, homescore, homesog, ot_reg,
                                      num, eventid, period, period_type, sit_code, timeinperiod, timeremaining,
                                       hometeamdefside, typecode, typekey, eventownerteamid,
                                      xcoord, ycoord, zone, descKey, homeplayer, awayplayer]],
                              columns = ['game_id','game_type', 'venue', 'start_time_utc', 'away_team_id','away_team_name',
                                        'away_team_abbrev','away_score','away_sog', 'home_team_id', 'home_team_name',
                                        'home_team_abbrev', 'home_score', 'home_sog','ot_reg', 'event_number',
                                        'event_id','period_number', 'period_type', 'situation_code', 'time_in_period',
                                        'time_remaining_in_period', 'home_team_defending_side',
                                        'event_type_code','event_type_key', 'event_owner_team_id',
                                        'x_coordinate', 'y_coordinate', 'zone', 'description_key',
                                        'home_player', 'away_player'])
            game_events.append(event_row)
            
        else:
            eventownerteamid = i['details']['eventOwnerTeamId']
            xcoord = 'NA'
            ycoord = 'NA'
            zone = 'NA'
            descKey = 'NA'
            homeplayer = 'NA'
            awayplayer = 'NA'
            event_row = pd.DataFrame([[gameid, gametype, venue, starttimeutc, awayid, awayname, awayabr, awayscore,
                                      awaysog, homeid, homename, homeabr, homescore, homesog, ot_reg,
                                      num, eventid, period, period_type, sit_code, timeinperiod, timeremaining,
                                       hometeamdefside, typecode, typekey, eventownerteamid,
                                      xcoord, ycoord, zone, descKey, homeplayer, awayplayer]],
                              columns = ['game_id','game_type', 'venue', 'start_time_utc', 'away_team_id','away_team_name',
                                        'away_team_abbrev','away_score','away_sog', 'home_team_id', 'home_team_name',
                                        'home_team_abbrev', 'home_score', 'home_sog','ot_reg', 'event_number',
                                        'event_id','period_number', 'period_type', 'situation_code', 'time_in_period',
                                        'time_remaining_in_period', 'home_team_defending_side',
                                        'event_type_code','event_type_key', 'event_owner_team_id',
                                        'x_coordinate', 'y_coordinate', 'zone', 'description_key',
                                        'home_player', 'away_player'])
            game_events.append(event_row)
            
    events_df = pd.concat(game_events)
    return events_df

In [ ]:
## game ids for 2021-2022

schedule_21_22 = get_schedule(20212022)
game_ids_21_22 = schedule_21_22['game_id'].to_list()

In [18]:
## game ids for 2022-2023

schedule_22_23 = get_schedule(20222023)
game_ids_22_23 = schedule_22_23['game_id'].to_list()

In [ ]:
## game ids for 2023-2024

schedule_23_24 = get_schedule(20232024)
game_ids_23_24 = schedule_23_24['game_id'].to_list()

In [42]:
## play-by-play scrape for 2021-2022

game_events_df_list_21_22 = []

for i in game_ids_21_22:
    game_events_df = get_data(i)
    game_events_df['season'] = '2021-2022'
    game_events_df_list_21_22.append(game_events_df)

In [ ]:
## play-by-play scrape for 2022-2023

game_events_df_list_22_23 = []

for i in game_ids_22_23:
    game_events_df = get_data(i)
    game_events_df['season'] = '2022-2023'
    game_events_df_list_22_23.append(game_events_df)

In [44]:
## play-by-play scrape for 2023-2024

game_events_df_list_23_24 = []

for i in game_ids_23_24:
    game_events_df = get_data(i)
    game_events_df['season'] = '2023-2024'
    game_events_df_list_23_24.append(game_events_df)

In [45]:
## writing df for 2021-2024 play-by-play

play_by_play_21_22 = pd.concat(game_events_df_list_21_22)
play_by_play_22_23 = pd.concat(game_events_df_list_22_23)
play_by_play_23_24 = pd.concat(game_events_df_list_23_24)
pbp_list = [play_by_play_21_22,play_by_play_22_23,play_by_play_23_24]
play_by_play_21_24 = pd.concat(pbp_list)
#play_by_play_21_24.to_csv('NHL_play_by_play_2021-2024_7_23_24.csv')

In [ ]:
## unique event types 

event_list = []

for i in game_ids_23_24:
    url = "https://api-web.nhle.com/v1/gamecenter/" + str(i) + "/play-by-play"
    response = requests.get(url)
    for j in response.json()['plays']:
        keys = j['typeDescKey']
        event_list.append(keys)
    
unique_events = list(set(event_list))
unique_events

In [47]:
## game ids for 2021-24

#play_by_play_21_24 = pd.read_csv('NHL_play_by_play_2021-2024_7_23_24.csv')
game_ids_21_24 = list(set(play_by_play_21_24['game_id'].to_list()))

In [ ]:
# get corsi and fenwick and game time 

play_by_play_df_list = []

for i in game_ids_21_24:
    df = play_by_play_21_24[play_by_play_21_24['game_id'] == i].sort_values(by=['event_number'])
    df = df[df['period_type'] != 'SO']
    df = df[df['period_type'] != 'OT']
    df['away_goal_cumsum'] = df.assign(goal=df['event_type_key'] == 'goal').where(df['event_owner_team_id'] == df['away_team_id']).goal.cumsum()
    df['home_goal_cumsum'] = df.assign(goal=df['event_type_key'] == 'goal').where(df['event_owner_team_id'] == df['home_team_id']).goal.cumsum()
    df['away_goal_cumsum'].iloc[0] = 0
    df['home_goal_cumsum'].iloc[0] = 0
    df['away_missed_shot_cumsum'] = df.assign(ms=df['event_type_key'] == 'missed-shot').where(df['event_owner_team_id'] == df['away_team_id']).ms.cumsum()
    df['home_missed_shot_cumsum'] = df.assign(ms=df['event_type_key'] == 'missed-shot').where(df['event_owner_team_id'] == df['home_team_id']).ms.cumsum()
    df['away_missed_shot_cumsum'].iloc[0] = 0
    df['home_missed_shot_cumsum'].iloc[0] = 0
    df['away_shot_on_goal_cumsum'] = df.assign(sog=df['event_type_key'] == 'shot-on-goal').where(df['event_owner_team_id'] == df['away_team_id']).sog.cumsum()
    df['home_shot_on_goal_cumsum'] = df.assign(sog=df['event_type_key'] == 'shot-on-goal').where(df['event_owner_team_id'] == df['home_team_id']).sog.cumsum()
    df['away_shot_on_goal_cumsum'].iloc[0] = 0
    df['home_shot_on_goal_cumsum'].iloc[0] = 0
    df['away_blocked_shot_cumsum'] = df.assign(bs=df['event_type_key'] == 'blocked-shot').where(df['event_owner_team_id'] == df['home_team_id']).bs.cumsum()
    df['home_blocked_shot_cumsum'] = df.assign(bs=df['event_type_key'] == 'blocked-shot').where(df['event_owner_team_id'] == df['away_team_id']).bs.cumsum()
    df['away_blocked_shot_cumsum'].iloc[0] = 0
    df['home_blocked_shot_cumsum'].iloc[0] = 0
    df[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']] = df[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']].fillna(method='ffill')
    df['away_corsi_cumsum'] = df['away_missed_shot_cumsum'] + df['away_shot_on_goal_cumsum'] + df['away_goal_cumsum'] + df['away_blocked_shot_cumsum']
    df['home_corsi_cumsum'] = df['home_missed_shot_cumsum'] + df['home_shot_on_goal_cumsum'] + df['home_goal_cumsum'] + df['home_blocked_shot_cumsum']
    df['away_fenwick_cumsum'] = df['away_missed_shot_cumsum'] + df['away_shot_on_goal_cumsum'] + df['away_goal_cumsum'] 
    df['home_fenwick_cumsum'] = df['home_missed_shot_cumsum'] + df['home_shot_on_goal_cumsum'] + df['home_goal_cumsum'] 
    df[['time_in_minutes', 'time_in_seconds']] = df['time_in_period'].str.split(':', expand=True)
    df['time_added_for_period'] = np.where(df['period_number'] == 1, 0, df['period_number']*20-20)
    df['game_time_passed_seconds'] = df.time_in_minutes.astype(float) * 60 + df.time_in_seconds.astype(float) + df['time_added_for_period']*60
    df['away_corsi_rate_per_second'] = df['away_corsi_cumsum']/df['game_time_passed_seconds']
    df['home_corsi_rate_per_second'] = df['home_corsi_cumsum']/df['game_time_passed_seconds']
    df['away_fenwick_rate_per_second'] = df['away_fenwick_cumsum']/df['game_time_passed_seconds']
    df['home_fenwick_rate_per_second'] = df['home_fenwick_cumsum']/df['game_time_passed_seconds']
    df[['away_corsi_rate_per_second','home_corsi_rate_per_second', 'away_fenwick_rate_per_second','home_fenwick_rate_per_second']] = df[['away_corsi_rate_per_second','home_corsi_rate_per_second', 'away_fenwick_rate_per_second','home_fenwick_rate_per_second']].fillna(value=0)
    play_by_play_df_list.append(df)
    
    

In [49]:
play_by_play_21_24[play_by_play_21_24['season'] == '2021-2022'].to_csv('nhl_pbp_22_9_2_24.csv')
play_by_play_21_24[play_by_play_21_24['season'] == '2022-2023'].to_csv('nhl_pbp_23_9_2_24.csv')
play_by_play_21_24[play_by_play_21_24['season'] == '2023-2024'].to_csv('nhl_pbp_24_9_2_24.csv')


In [ ]:
play_by_play_21_24 = play_by_play_21_24[play_by_play_21_24['away_goalie_in_net'] == 1]
play_by_play_21_24 = play_by_play_21_24[play_by_play_21_24['home_goalie_in_net'] == 1]
play_by_play_21_24 = play_by_play_21_24[play_by_play_21_24['period_type'] != 'SO']
play_by_play_21_24 = play_by_play_21_24[play_by_play_21_24['period_type'] != 'OT']


In [ ]:
play_by_play_21_24['strength'] = np.where(play_by_play_21_24['away_num_of_skaters'] == play_by_play_21_24['home_num_of_skaters'],
                                         'even',
                                         np.where(play_by_play_21_24['away_num_of_skaters'] > play_by_play_21_24['home_num_of_skaters'],
                                                 'away team powerplay',
                                                 'home team powerplay'))

play_by_play_21_24['home_team_advantage'] = play_by_play_21_24['home_num_of_skaters'] - play_by_play_21_24['away_num_of_skaters']


In [24]:
# group game stints by strength 

group_agg_df = []
split_df = []

for i in game_ids_21_24:
    df1 = play_by_play_21_24[play_by_play_21_24['game_id'] == i]
    df1 = df1.sort_values(by=['event_number'])
    df1 = df1.assign(difference=(diff:=df1.home_team_advantage.diff()), 
          condition=(gt1:=abs(diff).gt(0)), 
          grouper=gt1.cumsum())
    for j in range(max(df1['grouper']) + 1):
        df2 = df1[df1['grouper'] == j]
        df2['away_goal_cumsum'] = df2.assign(goal=df2['event_type_key'] == 'goal').where(df2['event_owner_team_id'] == df2['away_team_id']).goal.cumsum()
        df2['home_goal_cumsum'] = df2.assign(goal=df2['event_type_key'] == 'goal').where(df2['event_owner_team_id'] == df2['home_team_id']).goal.cumsum()
        df2['away_goal_cumsum'].iloc[0] = 0
        df2['home_goal_cumsum'].iloc[0] = 0
        df2['away_missed_shot_cumsum'] = df2.assign(ms=df2['event_type_key'] == 'missed-shot').where(df2['event_owner_team_id'] == df2['away_team_id']).ms.cumsum()
        df2['home_missed_shot_cumsum'] = df2.assign(ms=df2['event_type_key'] == 'missed-shot').where(df2['event_owner_team_id'] == df2['home_team_id']).ms.cumsum()
        df2['away_missed_shot_cumsum'].iloc[0] = 0
        df2['home_missed_shot_cumsum'].iloc[0] = 0
        df2['away_shot_on_goal_cumsum'] = df2.assign(sog=df2['event_type_key'] == 'shot-on-goal').where(df2['event_owner_team_id'] == df2['away_team_id']).sog.cumsum()
        df2['home_shot_on_goal_cumsum'] = df2.assign(sog=df2['event_type_key'] == 'shot-on-goal').where(df2['event_owner_team_id'] == df2['home_team_id']).sog.cumsum()
        df2['away_shot_on_goal_cumsum'].iloc[0] = 0
        df2['home_shot_on_goal_cumsum'].iloc[0] = 0
        df2['away_blocked_shot_cumsum'] = df2.assign(bs=df2['event_type_key'] == 'blocked-shot').where(df2['event_owner_team_id'] == df2['home_team_id']).bs.cumsum()
        df2['home_blocked_shot_cumsum'] = df2.assign(bs=df2['event_type_key'] == 'blocked-shot').where(df2['event_owner_team_id'] == df2['away_team_id']).bs.cumsum()
        df2['away_blocked_shot_cumsum'].iloc[0] = 0
        df2['home_blocked_shot_cumsum'].iloc[0] = 0
        df2[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']] = df2[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']].fillna(method='ffill')
        df2['away_corsi_cumsum'] = df2['away_missed_shot_cumsum'] + df2['away_shot_on_goal_cumsum'] + df2['away_goal_cumsum'] + df2['away_blocked_shot_cumsum']
        df2['home_corsi_cumsum'] = df2['home_missed_shot_cumsum'] + df2['home_shot_on_goal_cumsum'] + df2['home_goal_cumsum'] + df2['home_blocked_shot_cumsum']
        df2['away_fenwick_cumsum'] = df2['away_missed_shot_cumsum'] + df2['away_shot_on_goal_cumsum'] + df2['away_goal_cumsum'] 
        df2['home_fenwick_cumsum'] = df2['home_missed_shot_cumsum'] + df2['home_shot_on_goal_cumsum'] + df2['home_goal_cumsum'] 
        df2[['time_in_minutes', 'time_in_seconds']] = df2['time_in_period'].str.split(':', expand=True)
        df2['time_added_for_period'] = np.where(df2['period_number'] == 1, 0, df2['period_number']*20-20)
        df2['game_time_passed_seconds'] = df2.time_in_minutes.astype(float) * 60 + df2.time_in_seconds.astype(float) + df2['time_added_for_period']*60
        df2['away_corsi_rate_per_second'] = df2['away_corsi_cumsum']/(df2['game_time_passed_seconds'] - df2.iloc[0]['game_time_passed_seconds'])
        df2['home_corsi_rate_per_second'] = df2['home_corsi_cumsum']/(df2['game_time_passed_seconds'] - df2.iloc[0]['game_time_passed_seconds'])
        df2['away_fenwick_rate_per_second'] = df2['away_fenwick_cumsum']/(df2['game_time_passed_seconds'] - df2.iloc[0]['game_time_passed_seconds'])
        df2['home_fenwick_rate_per_second'] = df2['home_fenwick_cumsum']/(df2['game_time_passed_seconds'] - df2.iloc[0]['game_time_passed_seconds'])
        df2[['away_corsi_rate_per_second','home_corsi_rate_per_second', 'away_fenwick_rate_per_second','home_fenwick_rate_per_second']] = df2[['away_corsi_rate_per_second','home_corsi_rate_per_second', 'away_fenwick_rate_per_second','home_fenwick_rate_per_second']].fillna(value=0)
        df2['duration_seconds'] = df2.iloc[-1]['game_time_passed_seconds'] - df2.iloc[0]['game_time_passed_seconds']
        split_df.append(df2)
        df3 = pd.DataFrame([[df2.iloc[-1]['game_id'],
                            df2.iloc[-1]['grouper'],
                            df2.iloc[-1]['home_team_advantage'],
                            df2.iloc[-1]['strength'],
                            df2.iloc[0]['period_number'],
                            df2.iloc[-1]['duration_seconds'] ,
                            df2.iloc[-1]['away_goal_cumsum'],
                            df2.iloc[-1]['home_goal_cumsum'],
                            df2.iloc[-1]['away_missed_shot_cumsum'],
                            df2.iloc[-1]['home_missed_shot_cumsum'],
                            df2.iloc[-1]['away_shot_on_goal_cumsum'],
                            df2.iloc[-1]['home_shot_on_goal_cumsum'],
                            df2.iloc[-1]['away_blocked_shot_cumsum'],
                            df2.iloc[-1]['home_blocked_shot_cumsum'],
                            df2.iloc[-1]['away_corsi_cumsum'],
                            df2.iloc[-1]['home_corsi_cumsum'],
                            df2.iloc[-1]['away_fenwick_cumsum'],
                            df2.iloc[-1]['home_fenwick_cumsum'],
                             df2.iloc[-1]['away_corsi_rate_per_second'],
                             df2.iloc[-1]['home_corsi_rate_per_second'],
                             df2.iloc[-1]['away_fenwick_rate_per_second'],
                             df2.iloc[-1]['home_fenwick_rate_per_second']]],
                            columns = ['game_id', 'game_group_number', 'home_team_advantage',
                                      'strength_description', 'period_number_at_start', 
                                       'duration_seconds','away_goal_cumsum',
                                      'home_goal_cumsum', 'away_missed_shot_cumsum', 'home_missed_shot_cumsum',
                                      'away_shot_on_goal_cumsum', 'home_shot_on_goal_cumsum',
                                      'away_blocked_shot_cumsum', 'home_blocked_shot_cumsum',
                                      'away_corsi_cumsum', 'home_corsi_cumsum', 'away_fenwick_cumsum',
                                      'home_fenwick_cumsum', 'away_corsi_rate_per_second', 
                                      'home_corsi_rate_per_second', 'away_fenwick_rate_per_second',
                                      'home_fenwick_rate_per_second'])
        group_agg_df.append(df3)
        

In [ ]:
grouped_aggs = pd.concat(group_agg_df)
grouped_aggs

In [ ]:
grouped_aggs.to_csv('NHL_pbp_Strength_aggs.csv')

In [2]:
pbp22 = pd.read_csv('nhl_pbp_22_9_2_24.csv')
pbp23 = pd.read_csv('nhl_pbp_23_9_2_24.csv')
pbp24 = pd.read_csv('nhl_pbp_24_9_2_24.csv')
pbp_list = [pbp22, pbp23, pbp24]
play_by_play_21_24 = pd.concat(pbp_list)

game_ids_21_24 = list(set(play_by_play_21_24['game_id']))

In [3]:
# final fight dfs

fight_df_list = []

for i in game_ids_21_24:
    df = play_by_play_21_24[play_by_play_21_24['game_id'] == i].sort_values(by=['event_number'])
    df = df[df['period_type'] != 'SO']
    df = df[df['period_type'] != 'OT']
    df[['time_in_minutes', 'time_in_seconds']] = df['time_in_period'].str.split(':', expand=True)
    df['time_added_for_period'] = np.where(df['period_number'] == 1, 0, df['period_number']*20-20)
    df['game_time_passed_seconds'] = df.time_in_minutes.astype(float) * 60 + df.time_in_seconds.astype(float) + df['time_added_for_period']*60
    df = df.reset_index()
    num = 0
    for index, row in df.iterrows():
        if row['description_key'] == 'fighting':
            num=num+1
            # data frame before fight occurs
            df_before = df.iloc[:index]
            # data frame once fight occurs
            df_after = df.iloc[index:]
            df_after['game_fight_number'] = num

            # data frame cut off at 2, 5, 10 minutes, end of period, until next fight
            df_after_2_minutes = df_after[df_after['game_time_passed_seconds'] <= int(df_after.iloc[0]['game_time_passed_seconds']) + 120]
            df_after_5_minutes = df_after[df_after['game_time_passed_seconds'] <= int(df_after.iloc[0]['game_time_passed_seconds']) + 300]
            df_after_10_minutes = df_after[df_after['game_time_passed_seconds'] <= int(df_after.iloc[0]['game_time_passed_seconds']) + 600]
            df_until_eop = df_after[df_after['period_number'] == df_after.iloc[0]['period_number']]

            # data frame until next fight 
            df_after['num_fights'] = df_after.assign(fight=df_after['description_key'] == 'fighting').fight.cumsum()
            try:
                df_until_next_fight = df_after.loc[: df_after[(df_after['num_fights'] == 2)].index[0], :]
            except IndexError as error:
                pass
            
            # corsi and fenwick before the fight
            df_before['away_goal_cumsum'] = df_before.assign(goal=df_before['event_type_key'] == 'goal').where(df_before['event_owner_team_id'] == df_before['away_team_id']).goal.cumsum()
            df_before['home_goal_cumsum'] = df_before.assign(goal=df_before['event_type_key'] == 'goal').where(df_before['event_owner_team_id'] == df_before['home_team_id']).goal.cumsum()
            df_before['away_goal_cumsum'].iloc[0] = 0
            df_before['home_goal_cumsum'].iloc[0] = 0
            df_before['away_missed_shot_cumsum'] = df_before.assign(ms=df_before['event_type_key'] == 'missed-shot').where(df_before['event_owner_team_id'] == df_before['away_team_id']).ms.cumsum()
            df_before['home_missed_shot_cumsum'] = df_before.assign(ms=df_before['event_type_key'] == 'missed-shot').where(df_before['event_owner_team_id'] == df_before['home_team_id']).ms.cumsum()
            df_before['away_missed_shot_cumsum'].iloc[0] = 0
            df_before['home_missed_shot_cumsum'].iloc[0] = 0
            df_before['away_shot_on_goal_cumsum'] = df_before.assign(sog=df_before['event_type_key'] == 'shot-on-goal').where(df_before['event_owner_team_id'] == df_before['away_team_id']).sog.cumsum()
            df_before['home_shot_on_goal_cumsum'] = df_before.assign(sog=df_before['event_type_key'] == 'shot-on-goal').where(df_before['event_owner_team_id'] == df_before['home_team_id']).sog.cumsum()
            df_before['away_shot_on_goal_cumsum'].iloc[0] = 0
            df_before['home_shot_on_goal_cumsum'].iloc[0] = 0
            df_before['away_blocked_shot_cumsum'] = df_before.assign(bs=df_before['event_type_key'] == 'blocked-shot').where(df_before['event_owner_team_id'] == df_before['home_team_id']).bs.cumsum()
            df_before['home_blocked_shot_cumsum'] = df_before.assign(bs=df_before['event_type_key'] == 'blocked-shot').where(df_before['event_owner_team_id'] == df_before['away_team_id']).bs.cumsum()
            df_before['away_blocked_shot_cumsum'].iloc[0] = 0
            df_before['home_blocked_shot_cumsum'].iloc[0] = 0
            df_before[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']] = df_before[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']].fillna(method='ffill')
            df_before['away_corsi_cumsum'] = df_before['away_missed_shot_cumsum'] + df_before['away_shot_on_goal_cumsum'] + df_before['away_goal_cumsum'] + df_before['away_blocked_shot_cumsum']
            df_before['home_corsi_cumsum'] = df_before['home_missed_shot_cumsum'] + df_before['home_shot_on_goal_cumsum'] + df_before['home_goal_cumsum'] + df_before['home_blocked_shot_cumsum']
            df_before['away_fenwick_cumsum'] = df_before['away_missed_shot_cumsum'] + df_before['away_shot_on_goal_cumsum'] + df_before['away_goal_cumsum'] 
            df_before['home_fenwick_cumsum'] = df_before['home_missed_shot_cumsum'] + df_before['home_shot_on_goal_cumsum'] + df_before['home_goal_cumsum'] 
            game_time_passed_seconds_before = df_before.iloc[-1]['game_time_passed_seconds'] - df_before.iloc[0]['game_time_passed_seconds']
            df_before['away_corsi_rate_per_second'] = df_before['away_corsi_cumsum']/game_time_passed_seconds_before
            df_before['home_corsi_rate_per_second'] = df_before['home_corsi_cumsum']/game_time_passed_seconds_before
            df_before['away_fenwick_rate_per_second'] = df_before['away_fenwick_cumsum']/game_time_passed_seconds_before
            df_before['home_fenwick_rate_per_second'] = df_before['home_fenwick_cumsum']/game_time_passed_seconds_before
            df_before[['away_corsi_rate_per_second','home_corsi_rate_per_second', 'away_fenwick_rate_per_second','home_fenwick_rate_per_second']] = df_before[['away_corsi_rate_per_second','home_corsi_rate_per_second', 'away_fenwick_rate_per_second','home_fenwick_rate_per_second']].fillna(value=0)

            # corsi and fenwick rates at the time of the fight
            away_score_at_fight = df_before.iloc[-1]['away_goal_cumsum']
            home_score_at_fight = df_before.iloc[-1]['home_goal_cumsum']
            away_corsi_at_fight = df_before.iloc[-1]['away_corsi_rate_per_second']
            away_fenwick_at_fight = df_before.iloc[-1]['away_fenwick_rate_per_second']
            home_corsi_at_fight = df_before.iloc[-1]['home_corsi_rate_per_second']
            home_fenwick_at_fight = df_before.iloc[-1]['home_fenwick_rate_per_second']

            # corsi and fenwick 2 minutes after the fight
            df_after_2_minutes['away_goal_cumsum'] = df_after_2_minutes.assign(goal=df_after_2_minutes['event_type_key'] == 'goal').where(df_after_2_minutes['event_owner_team_id'] == df_after_2_minutes['away_team_id']).goal.cumsum()
            df_after_2_minutes['home_goal_cumsum'] = df_after_2_minutes.assign(goal=df_after_2_minutes['event_type_key'] == 'goal').where(df_after_2_minutes['event_owner_team_id'] == df_after_2_minutes['home_team_id']).goal.cumsum()
            df_after_2_minutes['away_goal_cumsum'].iloc[0] = 0
            df_after_2_minutes['home_goal_cumsum'].iloc[0] = 0
            df_after_2_minutes['away_missed_shot_cumsum'] = df_after_2_minutes.assign(ms=df_after_2_minutes['event_type_key'] == 'missed-shot').where(df_after_2_minutes['event_owner_team_id'] == df_after_2_minutes['away_team_id']).ms.cumsum()
            df_after_2_minutes['home_missed_shot_cumsum'] = df_after_2_minutes.assign(ms=df_after_2_minutes['event_type_key'] == 'missed-shot').where(df_after_2_minutes['event_owner_team_id'] == df_after_2_minutes['home_team_id']).ms.cumsum()
            df_after_2_minutes['away_missed_shot_cumsum'].iloc[0] = 0
            df_after_2_minutes['home_missed_shot_cumsum'].iloc[0] = 0
            df_after_2_minutes['away_shot_on_goal_cumsum'] = df_after_2_minutes.assign(sog=df_after_2_minutes['event_type_key'] == 'shot-on-goal').where(df_after_2_minutes['event_owner_team_id'] == df_after_2_minutes['away_team_id']).sog.cumsum()
            df_after_2_minutes['home_shot_on_goal_cumsum'] = df_after_2_minutes.assign(sog=df_after_2_minutes['event_type_key'] == 'shot-on-goal').where(df_after_2_minutes['event_owner_team_id'] == df_after_2_minutes['home_team_id']).sog.cumsum()
            df_after_2_minutes['away_shot_on_goal_cumsum'].iloc[0] = 0
            df_after_2_minutes['home_shot_on_goal_cumsum'].iloc[0] = 0
            df_after_2_minutes['away_blocked_shot_cumsum'] = df_after_2_minutes.assign(bs=df_after_2_minutes['event_type_key'] == 'blocked-shot').where(df_after_2_minutes['event_owner_team_id'] == df_after_2_minutes['home_team_id']).bs.cumsum()
            df_after_2_minutes['home_blocked_shot_cumsum'] = df_after_2_minutes.assign(bs=df_after_2_minutes['event_type_key'] == 'blocked-shot').where(df_after_2_minutes['event_owner_team_id'] == df_after_2_minutes['away_team_id']).bs.cumsum()
            df_after_2_minutes['away_blocked_shot_cumsum'].iloc[0] = 0
            df_after_2_minutes['home_blocked_shot_cumsum'].iloc[0] = 0
            df_after_2_minutes[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']] = df_after_2_minutes[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']].fillna(method='ffill')
            df_after_2_minutes['away_corsi_cumsum'] = df_after_2_minutes['away_missed_shot_cumsum'] + df_after_2_minutes['away_shot_on_goal_cumsum'] + df_after_2_minutes['away_goal_cumsum'] + df_after_2_minutes['away_blocked_shot_cumsum']
            df_after_2_minutes['home_corsi_cumsum'] = df_after_2_minutes['home_missed_shot_cumsum'] + df_after_2_minutes['home_shot_on_goal_cumsum'] + df_after_2_minutes['home_goal_cumsum'] + df_after_2_minutes['home_blocked_shot_cumsum']
            df_after_2_minutes['away_fenwick_cumsum'] = df_after_2_minutes['away_missed_shot_cumsum'] + df_after_2_minutes['away_shot_on_goal_cumsum'] + df_after_2_minutes['away_goal_cumsum'] 
            df_after_2_minutes['home_fenwick_cumsum'] = df_after_2_minutes['home_missed_shot_cumsum'] + df_after_2_minutes['home_shot_on_goal_cumsum'] + df_after_2_minutes['home_goal_cumsum'] 
            #game_time_passed_seconds_after_2 = df_after_2_minutes.iloc[-1]['game_time_passed_seconds'] - df_after_2_minutes.iloc[0]['game_time_passed_seconds']
            # corsi and fenwick rates 2 minutes after fight
            if row['game_time_passed_seconds'] <= 3480:
                away_corsi_after_2 = df_after_2_minutes.iloc[-1]['away_corsi_cumsum']/120
                away_fenwick_after_2 = df_after_2_minutes.iloc[-1]['away_fenwick_cumsum']/120
                home_corsi_after_2 = df_after_2_minutes.iloc[-1]['home_corsi_cumsum']/120
                home_fenwick_after_2 = df_after_2_minutes.iloc[-1]['home_fenwick_cumsum']/120
            else:
                away_corsi_after_2 = df_after_2_minutes.iloc[-1]['away_corsi_cumsum']/(3600-row['game_time_passed_seconds'])
                away_fenwick_after_2 = df_after_2_minutes.iloc[-1]['away_fenwick_cumsum']/(3600-row['game_time_passed_seconds'])
                home_corsi_after_2 = df_after_2_minutes.iloc[-1]['home_corsi_cumsum']/(3600-row['game_time_passed_seconds'])
                home_fenwick_after_2 = df_after_2_minutes.iloc[-1]['home_fenwick_cumsum']/(3600-row['game_time_passed_seconds'])

            # corsi and fenwick 5 minutes after the fight
            df_after_5_minutes['away_goal_cumsum'] = df_after_5_minutes.assign(goal=df_after_5_minutes['event_type_key'] == 'goal').where(df_after_5_minutes['event_owner_team_id'] == df_after_5_minutes['away_team_id']).goal.cumsum()
            df_after_5_minutes['home_goal_cumsum'] = df_after_5_minutes.assign(goal=df_after_5_minutes['event_type_key'] == 'goal').where(df_after_5_minutes['event_owner_team_id'] == df_after_5_minutes['home_team_id']).goal.cumsum()
            df_after_5_minutes['away_goal_cumsum'].iloc[0] = 0
            df_after_5_minutes['home_goal_cumsum'].iloc[0] = 0
            df_after_5_minutes['away_missed_shot_cumsum'] = df_after_5_minutes.assign(ms=df_after_5_minutes['event_type_key'] == 'missed-shot').where(df_after_5_minutes['event_owner_team_id'] == df_after_5_minutes['away_team_id']).ms.cumsum()
            df_after_5_minutes['home_missed_shot_cumsum'] = df_after_5_minutes.assign(ms=df_after_5_minutes['event_type_key'] == 'missed-shot').where(df_after_5_minutes['event_owner_team_id'] == df_after_5_minutes['home_team_id']).ms.cumsum()
            df_after_5_minutes['away_missed_shot_cumsum'].iloc[0] = 0
            df_after_5_minutes['home_missed_shot_cumsum'].iloc[0] = 0
            df_after_5_minutes['away_shot_on_goal_cumsum'] = df_after_5_minutes.assign(sog=df_after_5_minutes['event_type_key'] == 'shot-on-goal').where(df_after_5_minutes['event_owner_team_id'] == df_after_5_minutes['away_team_id']).sog.cumsum()
            df_after_5_minutes['home_shot_on_goal_cumsum'] = df_after_5_minutes.assign(sog=df_after_5_minutes['event_type_key'] == 'shot-on-goal').where(df_after_5_minutes['event_owner_team_id'] == df_after_5_minutes['home_team_id']).sog.cumsum()
            df_after_5_minutes['away_shot_on_goal_cumsum'].iloc[0] = 0
            df_after_5_minutes['home_shot_on_goal_cumsum'].iloc[0] = 0
            df_after_5_minutes['away_blocked_shot_cumsum'] = df_after_5_minutes.assign(bs=df_after_5_minutes['event_type_key'] == 'blocked-shot').where(df_after_5_minutes['event_owner_team_id'] == df_after_5_minutes['home_team_id']).bs.cumsum()
            df_after_5_minutes['home_blocked_shot_cumsum'] = df_after_5_minutes.assign(bs=df_after_5_minutes['event_type_key'] == 'blocked-shot').where(df_after_5_minutes['event_owner_team_id'] == df_after_5_minutes['away_team_id']).bs.cumsum()
            df_after_5_minutes['away_blocked_shot_cumsum'].iloc[0] = 0
            df_after_5_minutes['home_blocked_shot_cumsum'].iloc[0] = 0
            df_after_5_minutes[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']] = df_after_5_minutes[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']].fillna(method='ffill')
            df_after_5_minutes['away_corsi_cumsum'] = df_after_5_minutes['away_missed_shot_cumsum'] + df_after_5_minutes['away_shot_on_goal_cumsum'] + df_after_5_minutes['away_goal_cumsum'] + df_after_5_minutes['away_blocked_shot_cumsum']
            df_after_5_minutes['home_corsi_cumsum'] = df_after_5_minutes['home_missed_shot_cumsum'] + df_after_5_minutes['home_shot_on_goal_cumsum'] + df_after_5_minutes['home_goal_cumsum'] + df_after_5_minutes['home_blocked_shot_cumsum']
            df_after_5_minutes['away_fenwick_cumsum'] = df_after_5_minutes['away_missed_shot_cumsum'] + df_after_5_minutes['away_shot_on_goal_cumsum'] + df_after_5_minutes['away_goal_cumsum'] 
            df_after_5_minutes['home_fenwick_cumsum'] = df_after_5_minutes['home_missed_shot_cumsum'] + df_after_5_minutes['home_shot_on_goal_cumsum'] + df_after_5_minutes['home_goal_cumsum'] 
            #game_time_passed_seconds_after_5 = df_after_5_minutes.iloc[-1]['game_time_passed_seconds'] - df_after_5_minutes.iloc[0]['game_time_passed_seconds']
            # corsi and fenwick rates after 5
            if row['game_time_passed_seconds'] <= 3300:
                away_corsi_after_5 = df_after_5_minutes.iloc[-1]['away_corsi_cumsum']/300
                away_fenwick_after_5 = df_after_5_minutes.iloc[-1]['away_fenwick_cumsum']/300
                home_corsi_after_5 = df_after_5_minutes.iloc[-1]['home_corsi_cumsum']/300
                home_fenwick_after_5 = df_after_5_minutes.iloc[-1]['home_fenwick_cumsum']/300
            else:
                away_corsi_after_5 = df_after_5_minutes.iloc[-1]['away_corsi_cumsum']/(3600-row['game_time_passed_seconds'])
                away_fenwick_after_5 = df_after_5_minutes.iloc[-1]['away_fenwick_cumsum']/(3600-row['game_time_passed_seconds'])
                home_corsi_after_5 = df_after_5_minutes.iloc[-1]['home_corsi_cumsum']/(3600-row['game_time_passed_seconds'])
                home_fenwick_after_5 = df_after_5_minutes.iloc[-1]['home_fenwick_cumsum']/(3600-row['game_time_passed_seconds'])



            # corsi and fenwick 10 minutes after the fight
            df_after_10_minutes['away_goal_cumsum'] = df_after_10_minutes.assign(goal=df_after_10_minutes['event_type_key'] == 'goal').where(df_after_10_minutes['event_owner_team_id'] == df_after_10_minutes['away_team_id']).goal.cumsum()
            df_after_10_minutes['home_goal_cumsum'] = df_after_10_minutes.assign(goal=df_after_10_minutes['event_type_key'] == 'goal').where(df_after_10_minutes['event_owner_team_id'] == df_after_10_minutes['home_team_id']).goal.cumsum()
            df_after_10_minutes['away_goal_cumsum'].iloc[0] = 0
            df_after_10_minutes['home_goal_cumsum'].iloc[0] = 0
            df_after_10_minutes['away_missed_shot_cumsum'] = df_after_10_minutes.assign(ms=df_after_10_minutes['event_type_key'] == 'missed-shot').where(df_after_10_minutes['event_owner_team_id'] == df_after_10_minutes['away_team_id']).ms.cumsum()
            df_after_10_minutes['home_missed_shot_cumsum'] = df_after_10_minutes.assign(ms=df_after_10_minutes['event_type_key'] == 'missed-shot').where(df_after_10_minutes['event_owner_team_id'] == df_after_10_minutes['home_team_id']).ms.cumsum()
            df_after_10_minutes['away_missed_shot_cumsum'].iloc[0] = 0
            df_after_10_minutes['home_missed_shot_cumsum'].iloc[0] = 0
            df_after_10_minutes['away_shot_on_goal_cumsum'] = df_after_10_minutes.assign(sog=df_after_10_minutes['event_type_key'] == 'shot-on-goal').where(df_after_10_minutes['event_owner_team_id'] == df_after_10_minutes['away_team_id']).sog.cumsum()
            df_after_10_minutes['home_shot_on_goal_cumsum'] = df_after_10_minutes.assign(sog=df_after_10_minutes['event_type_key'] == 'shot-on-goal').where(df_after_10_minutes['event_owner_team_id'] == df_after_10_minutes['home_team_id']).sog.cumsum()
            df_after_10_minutes['away_shot_on_goal_cumsum'].iloc[0] = 0
            df_after_10_minutes['home_shot_on_goal_cumsum'].iloc[0] = 0
            df_after_10_minutes['away_blocked_shot_cumsum'] = df_after_10_minutes.assign(bs=df_after_10_minutes['event_type_key'] == 'blocked-shot').where(df_after_10_minutes['event_owner_team_id'] == df_after_10_minutes['home_team_id']).bs.cumsum()
            df_after_10_minutes['home_blocked_shot_cumsum'] = df_after_10_minutes.assign(bs=df_after_10_minutes['event_type_key'] == 'blocked-shot').where(df_after_10_minutes['event_owner_team_id'] == df_after_10_minutes['away_team_id']).bs.cumsum()
            df_after_10_minutes['away_blocked_shot_cumsum'].iloc[0] = 0
            df_after_10_minutes['home_blocked_shot_cumsum'].iloc[0] = 0
            df_after_10_minutes[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']] = df_after_10_minutes[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']].fillna(method='ffill')
            df_after_10_minutes['away_corsi_cumsum'] = df_after_10_minutes['away_missed_shot_cumsum'] + df_after_10_minutes['away_shot_on_goal_cumsum'] + df_after_10_minutes['away_goal_cumsum'] + df_after_10_minutes['away_blocked_shot_cumsum']
            df_after_10_minutes['home_corsi_cumsum'] = df_after_10_minutes['home_missed_shot_cumsum'] + df_after_10_minutes['home_shot_on_goal_cumsum'] + df_after_10_minutes['home_goal_cumsum'] + df_after_10_minutes['home_blocked_shot_cumsum']
            df_after_10_minutes['away_fenwick_cumsum'] = df_after_10_minutes['away_missed_shot_cumsum'] + df_after_10_minutes['away_shot_on_goal_cumsum'] + df_after_10_minutes['away_goal_cumsum'] 
            df_after_10_minutes['home_fenwick_cumsum'] = df_after_10_minutes['home_missed_shot_cumsum'] + df_after_10_minutes['home_shot_on_goal_cumsum'] + df_after_10_minutes['home_goal_cumsum'] 
            #game_time_passed_seconds_after_10 = df_after_10_minutes.iloc[-1]['game_time_passed_seconds'] - df_after_10_minutes.iloc[0]['game_time_passed_seconds']
            if row['game_time_passed_seconds'] <= 3000:
                away_corsi_after_10 = df_after_10_minutes.iloc[-1]['away_corsi_cumsum']/600
                away_fenwick_after_10 = df_after_10_minutes.iloc[-1]['away_fenwick_cumsum']/600
                home_corsi_after_10 = df_after_10_minutes.iloc[-1]['home_corsi_cumsum']/600
                home_fenwick_after_10 = df_after_10_minutes.iloc[-1]['home_fenwick_cumsum']/600
            else:
                away_corsi_after_10 = df_after_10_minutes.iloc[-1]['away_corsi_cumsum']/(3600-row['game_time_passed_seconds'])
                away_fenwick_after_10 = df_after_10_minutes.iloc[-1]['away_fenwick_cumsum']/(3600-row['game_time_passed_seconds'])
                home_corsi_after_10 = df_after_10_minutes.iloc[-1]['home_corsi_cumsum']/(3600-row['game_time_passed_seconds'])
                home_fenwick_after_10 = df_after_10_minutes.iloc[-1]['home_fenwick_cumsum']/(3600-row['game_time_passed_seconds'])



             # corsi and fenwick until end of period after the fight
            df_until_eop['away_goal_cumsum'] = df_until_eop.assign(goal=df_until_eop['event_type_key'] == 'goal').where(df_until_eop['event_owner_team_id'] == df_until_eop['away_team_id']).goal.cumsum()
            df_until_eop['home_goal_cumsum'] = df_until_eop.assign(goal=df_until_eop['event_type_key'] == 'goal').where(df_until_eop['event_owner_team_id'] == df_until_eop['home_team_id']).goal.cumsum()
            df_until_eop['away_goal_cumsum'].iloc[0] = 0
            df_until_eop['home_goal_cumsum'].iloc[0] = 0
            df_until_eop['away_missed_shot_cumsum'] = df_until_eop.assign(ms=df_until_eop['event_type_key'] == 'missed-shot').where(df_until_eop['event_owner_team_id'] == df_until_eop['away_team_id']).ms.cumsum()
            df_until_eop['home_missed_shot_cumsum'] = df_until_eop.assign(ms=df_until_eop['event_type_key'] == 'missed-shot').where(df_until_eop['event_owner_team_id'] == df_until_eop['home_team_id']).ms.cumsum()
            df_until_eop['away_missed_shot_cumsum'].iloc[0] = 0
            df_until_eop['home_missed_shot_cumsum'].iloc[0] = 0
            df_until_eop['away_shot_on_goal_cumsum'] = df_until_eop.assign(sog=df_until_eop['event_type_key'] == 'shot-on-goal').where(df_until_eop['event_owner_team_id'] == df_until_eop['away_team_id']).sog.cumsum()
            df_until_eop['home_shot_on_goal_cumsum'] = df_until_eop.assign(sog=df_until_eop['event_type_key'] == 'shot-on-goal').where(df_until_eop['event_owner_team_id'] == df_until_eop['home_team_id']).sog.cumsum()
            df_until_eop['away_shot_on_goal_cumsum'].iloc[0] = 0
            df_until_eop['home_shot_on_goal_cumsum'].iloc[0] = 0
            df_until_eop['away_blocked_shot_cumsum'] = df_until_eop.assign(bs=df_until_eop['event_type_key'] == 'blocked-shot').where(df_until_eop['event_owner_team_id'] == df_until_eop['home_team_id']).bs.cumsum()
            df_until_eop['home_blocked_shot_cumsum'] = df_until_eop.assign(bs=df_until_eop['event_type_key'] == 'blocked-shot').where(df_until_eop['event_owner_team_id'] == df_until_eop['away_team_id']).bs.cumsum()
            df_until_eop['away_blocked_shot_cumsum'].iloc[0] = 0
            df_until_eop['home_blocked_shot_cumsum'].iloc[0] = 0
            df_until_eop[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']] = df_until_eop[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']].fillna(method='ffill')
            df_until_eop['away_corsi_cumsum'] = df_until_eop['away_missed_shot_cumsum'] + df_until_eop['away_shot_on_goal_cumsum'] + df_until_eop['away_goal_cumsum'] + df_until_eop['away_blocked_shot_cumsum']
            df_until_eop['home_corsi_cumsum'] = df_until_eop['home_missed_shot_cumsum'] + df_until_eop['home_shot_on_goal_cumsum'] + df_until_eop['home_goal_cumsum'] + df_until_eop['home_blocked_shot_cumsum']
            df_until_eop['away_fenwick_cumsum'] = df_until_eop['away_missed_shot_cumsum'] + df_until_eop['away_shot_on_goal_cumsum'] + df_until_eop['away_goal_cumsum'] 
            df_until_eop['home_fenwick_cumsum'] = df_until_eop['home_missed_shot_cumsum'] + df_until_eop['home_shot_on_goal_cumsum'] + df_until_eop['home_goal_cumsum'] 
            #game_time_passed_seconds_until_eop = df_until_eop.iloc[-1]['game_time_passed_seconds'] - df_until_eop.iloc[0]['game_time_passed_seconds']
            
            # corsi and fenwick rates until end of period after the fight
            away_corsi_until_eop = df_until_eop.iloc[-1]['away_corsi_cumsum']/((1200*row['period_number'])-row['game_time_passed_seconds'])
            away_fenwick_until_eop = df_until_eop.iloc[-1]['away_fenwick_cumsum']/((1200*row['period_number'])-row['game_time_passed_seconds'])
            home_corsi_until_eop = df_until_eop.iloc[-1]['home_corsi_cumsum']/((1200*row['period_number'])-row['game_time_passed_seconds'])
            home_fenwick_until_eop = df_until_eop.iloc[-1]['home_fenwick_cumsum']/((1200*row['period_number'])-row['game_time_passed_seconds'])


               # corsi and fenwick until end of game after the fight
            df_after['away_goal_cumsum'] = df_after.assign(goal=df_after['event_type_key'] == 'goal').where(df_after['event_owner_team_id'] == df_after['away_team_id']).goal.cumsum()
            df_after['home_goal_cumsum'] = df_after.assign(goal=df_after['event_type_key'] == 'goal').where(df_after['event_owner_team_id'] == df_after['home_team_id']).goal.cumsum()
            df_after['away_goal_cumsum'].iloc[0] = 0
            df_after['home_goal_cumsum'].iloc[0] = 0
            df_after['away_missed_shot_cumsum'] = df_after.assign(ms=df_after['event_type_key'] == 'missed-shot').where(df_after['event_owner_team_id'] == df_after['away_team_id']).ms.cumsum()
            df_after['home_missed_shot_cumsum'] = df_after.assign(ms=df_after['event_type_key'] == 'missed-shot').where(df_after['event_owner_team_id'] == df_after['home_team_id']).ms.cumsum()
            df_after['away_missed_shot_cumsum'].iloc[0] = 0
            df_after['home_missed_shot_cumsum'].iloc[0] = 0
            df_after['away_shot_on_goal_cumsum'] = df_after.assign(sog=df_after['event_type_key'] == 'shot-on-goal').where(df_after['event_owner_team_id'] == df_after['away_team_id']).sog.cumsum()
            df_after['home_shot_on_goal_cumsum'] = df_after.assign(sog=df_after['event_type_key'] == 'shot-on-goal').where(df_after['event_owner_team_id'] == df_after['home_team_id']).sog.cumsum()
            df_after['away_shot_on_goal_cumsum'].iloc[0] = 0
            df_after['home_shot_on_goal_cumsum'].iloc[0] = 0
            df_after['away_blocked_shot_cumsum'] = df_after.assign(bs=df_after['event_type_key'] == 'blocked-shot').where(df_after['event_owner_team_id'] == df_after['home_team_id']).bs.cumsum()
            df_after['home_blocked_shot_cumsum'] = df_after.assign(bs=df_after['event_type_key'] == 'blocked-shot').where(df_after['event_owner_team_id'] == df_after['away_team_id']).bs.cumsum()
            df_after['away_blocked_shot_cumsum'].iloc[0] = 0
            df_after['home_blocked_shot_cumsum'].iloc[0] = 0
            df_after[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']] = df_after[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']].fillna(method='ffill')
            df_after['away_corsi_cumsum'] = df_after['away_missed_shot_cumsum'] + df_after['away_shot_on_goal_cumsum'] + df_after['away_goal_cumsum'] + df_after['away_blocked_shot_cumsum']
            df_after['home_corsi_cumsum'] = df_after['home_missed_shot_cumsum'] + df_after['home_shot_on_goal_cumsum'] + df_after['home_goal_cumsum'] + df_after['home_blocked_shot_cumsum']
            df_after['away_fenwick_cumsum'] = df_after['away_missed_shot_cumsum'] + df_after['away_shot_on_goal_cumsum'] + df_after['away_goal_cumsum'] 
            df_after['home_fenwick_cumsum'] = df_after['home_missed_shot_cumsum'] + df_after['home_shot_on_goal_cumsum'] + df_after['home_goal_cumsum'] 
            # game_time_passed_seconds_until_eog = df_after.iloc[-1]['game_time_passed_seconds'] - df_after.iloc[0]['game_time_passed_seconds']
           
            # coris and fenwick until end of game
            away_corsi_until_eog = df_after.iloc[-1]['away_corsi_cumsum']/(3600-row['game_time_passed_seconds'])
            away_fenwick_until_eog = df_after.iloc[-1]['away_fenwick_cumsum']/(3600-row['game_time_passed_seconds'])
            home_corsi_until_eog = df_after.iloc[-1]['home_corsi_cumsum']/(3600-row['game_time_passed_seconds'])
            home_fenwick_until_eog = df_after.iloc[-1]['home_fenwick_cumsum']/(3600-row['game_time_passed_seconds'])



             # corsi and fenwick until next fight after the fight
            df_until_next_fight['away_goal_cumsum'] = df_until_next_fight.assign(goal=df_until_next_fight['event_type_key'] == 'goal').where(df_until_next_fight['event_owner_team_id'] == df_until_next_fight['away_team_id']).goal.cumsum()
            df_until_next_fight['home_goal_cumsum'] = df_until_next_fight.assign(goal=df_until_next_fight['event_type_key'] == 'goal').where(df_until_next_fight['event_owner_team_id'] == df_until_next_fight['home_team_id']).goal.cumsum()
            df_until_next_fight['away_goal_cumsum'].iloc[0] = 0
            df_until_next_fight['home_goal_cumsum'].iloc[0] = 0
            df_until_next_fight['away_missed_shot_cumsum'] = df_until_next_fight.assign(ms=df_until_next_fight['event_type_key'] == 'missed-shot').where(df_until_next_fight['event_owner_team_id'] == df_until_next_fight['away_team_id']).ms.cumsum()
            df_until_next_fight['home_missed_shot_cumsum'] = df_until_next_fight.assign(ms=df_until_next_fight['event_type_key'] == 'missed-shot').where(df_until_next_fight['event_owner_team_id'] == df_until_next_fight['home_team_id']).ms.cumsum()
            df_until_next_fight['away_missed_shot_cumsum'].iloc[0] = 0
            df_until_next_fight['home_missed_shot_cumsum'].iloc[0] = 0
            df_until_next_fight['away_shot_on_goal_cumsum'] = df_until_next_fight.assign(sog=df_until_next_fight['event_type_key'] == 'shot-on-goal').where(df_until_next_fight['event_owner_team_id'] == df_until_next_fight['away_team_id']).sog.cumsum()
            df_until_next_fight['home_shot_on_goal_cumsum'] = df_until_next_fight.assign(sog=df_until_next_fight['event_type_key'] == 'shot-on-goal').where(df_until_next_fight['event_owner_team_id'] == df_until_next_fight['home_team_id']).sog.cumsum()
            df_until_next_fight['away_shot_on_goal_cumsum'].iloc[0] = 0
            df_until_next_fight['home_shot_on_goal_cumsum'].iloc[0] = 0
            df_until_next_fight['away_blocked_shot_cumsum'] = df_until_next_fight.assign(bs=df_until_next_fight['event_type_key'] == 'blocked-shot').where(df_until_next_fight['event_owner_team_id'] == df_until_next_fight['home_team_id']).bs.cumsum()
            df_until_next_fight['home_blocked_shot_cumsum'] = df_until_next_fight.assign(bs=df_until_next_fight['event_type_key'] == 'blocked-shot').where(df_until_next_fight['event_owner_team_id'] == df_until_next_fight['away_team_id']).bs.cumsum()
            df_until_next_fight['away_blocked_shot_cumsum'].iloc[0] = 0
            df_until_next_fight['home_blocked_shot_cumsum'].iloc[0] = 0
            df_until_next_fight[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']] = df_until_next_fight[['away_goal_cumsum','home_goal_cumsum','away_missed_shot_cumsum','home_missed_shot_cumsum','away_shot_on_goal_cumsum','home_shot_on_goal_cumsum','away_blocked_shot_cumsum','home_blocked_shot_cumsum']].fillna(method='ffill')
            df_until_next_fight['away_corsi_cumsum'] = df_until_next_fight['away_missed_shot_cumsum'] + df_until_next_fight['away_shot_on_goal_cumsum'] + df_until_next_fight['away_goal_cumsum'] + df_until_next_fight['away_blocked_shot_cumsum']
            df_until_next_fight['home_corsi_cumsum'] = df_until_next_fight['home_missed_shot_cumsum'] + df_until_next_fight['home_shot_on_goal_cumsum'] + df_until_next_fight['home_goal_cumsum'] + df_until_next_fight['home_blocked_shot_cumsum']
            df_until_next_fight['away_fenwick_cumsum'] = df_until_next_fight['away_missed_shot_cumsum'] + df_until_next_fight['away_shot_on_goal_cumsum'] + df_until_next_fight['away_goal_cumsum'] 
            df_until_next_fight['home_fenwick_cumsum'] = df_until_next_fight['home_missed_shot_cumsum'] + df_until_next_fight['home_shot_on_goal_cumsum'] + df_until_next_fight['home_goal_cumsum'] 
            #game_time_passed_seconds_until_next_fight = df_until_next_fight.iloc[-1]['game_time_passed_seconds'] - df_until_next_fight.iloc[0]['game_time_passed_seconds']
         
            # corsi and fenwick rates until next fight after the fight
            away_corsi_until_next_fight = df_until_next_fight.iloc[-1]['away_corsi_cumsum']/(df_until_next_fight.iloc[-1]['game_time_passed_seconds'] - row['game_time_passed_seconds'])
            away_fenwick_until_next_fight = df_until_next_fight.iloc[-1]['away_fenwick_cumsum']/(df_until_next_fight.iloc[-1]['game_time_passed_seconds'] - row['game_time_passed_seconds'])
            home_corsi_until_next_fight = df_until_next_fight.iloc[-1]['home_corsi_cumsum']/(df_until_next_fight.iloc[-1]['game_time_passed_seconds'] - row['game_time_passed_seconds'])
            home_fenwick_until_next_fight = df_until_next_fight.iloc[-1]['home_fenwick_cumsum']/(df_until_next_fight.iloc[-1]['game_time_passed_seconds'] - row['game_time_passed_seconds'])


            # final df with all stats
            fight_df = pd.DataFrame([[row['game_id'], row['period_number'], num, row['game_time_passed_seconds'], 
                                      away_score_at_fight, home_score_at_fight, row['home_player'], row['away_player'],
                                     away_corsi_at_fight, away_fenwick_at_fight, home_corsi_at_fight, home_fenwick_at_fight,
                                     away_corsi_after_2, away_fenwick_after_2, home_corsi_after_2, home_fenwick_after_2,
                                     away_corsi_after_5, away_fenwick_after_5, home_corsi_after_5, home_fenwick_after_5,
                                     away_corsi_after_10, away_fenwick_after_10, home_corsi_after_10, home_fenwick_after_10,
                                     away_corsi_until_eop, away_fenwick_until_eop, home_corsi_until_eop, home_fenwick_until_eop,
                                     away_corsi_until_eog, away_fenwick_until_eog, home_corsi_until_eog, home_fenwick_until_eog,
                                     away_corsi_until_next_fight, away_fenwick_until_next_fight, home_corsi_until_next_fight, home_fenwick_until_next_fight]],
                                  columns = ['game_id', 'game_period_number', 'game_fight_number', 
                                            'game_time_of_fight', 'away_score_at_fight', 'home_score_at_fight',
                                             'home_fighter_id', 'away_fighter_id',
                                             'away_corsi_at_fight', 'away_fenwick_at_fight', 'home_corsi_at_fight', 
                                             'home_fenwick_at_fight',
                                     'away_corsi_after_2', 'away_fenwick_after_2', 'home_corsi_after_2', 'home_fenwick_after_2',
                                     'away_corsi_after_5', 'away_fenwick_after_5', 'home_corsi_after_5', 'home_fenwick_after_5',
                                     'away_corsi_after_10', 'away_fenwick_after_10', 'home_corsi_after_10', 'home_fenwick_after_10',
                                     'away_corsi_until_eop', 'away_fenwick_until_eop', 'home_corsi_until_eop', 'home_fenwick_until_eop',
                                     'away_corsi_until_eog', 'away_fenwick_until_eog', 'home_corsi_until_eog', 'home_fenwick_until_eog',
                                    'away_corsi_until_next_fight', 'away_fenwick_until_next_fight', 'home_corsi_until_next_fight', 'home_fenwick_until_next_fight'])

            if fight_df.iloc[0]['game_fight_number'] % 2 == 0:
                fight_df['game_fight_number'] = fight_df['game_fight_number'] / 2
                fight_df_list.append(fight_df)




In [5]:
fights = pd.concat(fight_df_list)

fights.sort_values(by=['game_id', 'game_time_of_fight']).to_csv('NHL_pbp_Fight_aggs3.csv')

#fights2.sort_values(by=['game_id']).to_csv('NHL_pbp_Fight_aggs2.csv')

In [ ]:
# get shifts data

pbp22 = pd.read_csv('nhl_pbp_22_9_2_24.csv')
pbp23 = pd.read_csv('nhl_pbp_23_9_2_24.csv')
pbp24 = pd.read_csv('nhl_pbp_24_9_2_24.csv')
pbp_list = [pbp22, pbp23, pbp24]
play_by_play_21_24 = pd.concat(pbp_list)

game_ids_21_24 = list(set(play_by_play_21_24['game_id']))

shifts_list = []

for i in game_ids_21_24:
    url = 'https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId=' + str(i)
    response = requests.get(url)
    for j in response.json()['data']:
        first = j['firstName']
        last = j['lastName']
        pid = j['playerId']
        shiftnum = j['shiftNumber']
        start = j['startTime']
        end = j['endTime']
        team = j['teamAbbrev']
        period = j['period']
        shifts = pd.DataFrame([[i, first, last, shiftnum, period, start, end, team]],
                             columns = ['game_id','firstname', 'lastname', 'shift_number', 'period',
                                       'start_time', 'end_time', 'team'])
        shifts_list.append(shifts)
    
shifts_df = pd.concat(shifts_list)
shifts_df.sort_values(['game_id','period', 'start_time', 'team'])

In [ ]:
shifts_df[['time_in_minutes_start', 'time_in_seconds_start']] = shifts_df['start_time'].str.split(':', expand=True)
shifts_df[['time_in_minutes_end', 'time_in_seconds_end']] = shifts_df['end_time'].str.split(':', expand=True)
shifts_df['time_added_for_period'] = np.where(shifts_df['period'] == 1, 0, shifts_df['period']*20-20)
shifts_df['start_time_sec'] = shifts_df.time_in_minutes_start.astype(float) * 60 + shifts_df.time_in_seconds_start.astype(float) + shifts_df['time_added_for_period']*60
shifts_df['end_time_sec'] = shifts_df.time_in_minutes_end.astype(float) * 60 + shifts_df.time_in_seconds_end.astype(float) + shifts_df['time_added_for_period']*60
shifts_df[['game_id', 'team', 'firstname', 'lastname', 'shift_number', 'period', 'start_time', 
           'start_time_sec','end_time', 'end_time_sec']].sort_values(['game_id','period', 'start_time', 'team']).to_csv('NHL_Shifts.csv')

In [ ]:
pbp22 = pd.read_csv('nhl_pbp_22_9_2_24.csv')
pbp23 = pd.read_csv('nhl_pbp_23_9_2_24.csv')
pbp24 = pd.read_csv('nhl_pbp_24_9_2_24.csv')
pbp_list = [pbp22, pbp23, pbp24]

updated = []

for i in pbp_list:
    i[['time_in_minutes', 'time_in_seconds']] = i['time_in_period'].str.split(':', expand=True)
    i['time_added_for_period'] = np.where(i['period_number'] == 1, 0, i['period_number']*20-20)
    i['game_time_passed_seconds'] = i.time_in_minutes.astype(float) * 60 + i.time_in_seconds.astype(float) + i['time_added_for_period']*60
    updated.append(i)
    
updated[0].to_csv('NHL_PBP_2021_22_9_7_24.csv')
updated[1].to_csv('NHL_PBP_2022_23_9_7_24.csv')
updated[2].to_csv('NHL_PBP_2023_24_9_7_24.csv')

In [ ]:
p22 = pd.read_csv('NHL_PBP_2021_22_9_7_24.csv')
p23 = pd.read_csv('NHL_PBP_2022_23_9_7_24.csv')
p24 = pd.read_csv('NHL_PBP_2023_24_9_7_24.csv')

plist = [p22,p23,p24]

pdf = pd.concat(plist)

pdf['away_total_players'] = pdf['away_goalie'] + pdf['away_skaters']
pdf['home_total_players'] = pdf['home_goalie'] + pdf['home_skaters']

pdf['home_advantage'] = pdf['home_total_players'] - pdf['away_total_players']

pdf.to_csv('NHL_pbp_2021-24_9_7_24.csv')
